### Assignment - 3 Web Scraping using selenium

#### Question 1 - Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [1]:
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
#load the drivers and URL
driver=webdriver.Chrome(r"C:\Users\rahsh\Downloads\chromedriver_win32\chromedriver.exe")
driver.get('http://amazon.in/')
driver.implicitly_wait(5)

product_vertical=input("Enter product vertical that needs to be searched:")
driver.find_element_by_id("twotabsearchtextbox").send_keys(product_vertical)
driver.find_element_by_id("nav-search-submit-button").click()

Enter product vertical that needs to be searched:Laptop


#### Question 2 - In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
#create a function to get the details of all products
urls=[]
for k in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break
        
        
# Let's create empty list
brand_name=[]
name_of_the_product=[]
rating=[]
no_of_ratings=[]
price=[]
expected_delivery=[]
return_exchange=[]
other_details=[]
availability=[]
product_url=[]

# checking all URL one by one
for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5)
    
    
#checking the brand name
    try: brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: brand_name.append('-')
    driver.implicitly_wait(5)
    
#checking name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        name_of_the_product.append(string)
    except: name_of_the_product.append('-')
    driver.implicitly_wait(5)
    
#checking rating of the product
    try: rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rating.append('-')
    driver.implicitly_wait(5)
    
#check number of ratings
    try: no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: no_of_ratings.append('-')
    driver.implicitly_wait(5)
    
#check the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
#check details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                return_exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            return_exchange.append('-')
    except: return_exchange.append('-')
    driver.implicitly_wait(5)
    
#scrape the expected delivery date
    try: expected_delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: expected_delivery.append('-')
    driver.implicitly_wait(5)
    
#scrape the availablity of the product
    try: 
        try: availability.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: availability.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: availability.append('-')
    driver.implicitly_wait(5)
    
#scrape other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other_details.append(details[0])
    except: other_details.append('-')
    driver.implicitly_wait(5)

#create a DataFrame for our data
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':name_of_the_product,'Rating':rating,'No. of Ratings':no_of_ratings,'Price':price,
             'Return/Exchange':return_exchange,'Expected Delivery':expected_delivery,'Availability':availability,'Other details':other_details,
             'Product URL':product_url})

#save the dataframe in csv file
amazon.to_csv('products.csv')

In [4]:
amazon

,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,LG,Gram 10th Gen Intel Core i7-1065G7 17-inch IP...,4.5 out of 5 stars,15 ratings,"₹ 99,900.00",7 Days Replacement,"Monday, March 1",In stock.,"Brand LG---Manufacturer LG, 1800 315 9999 / 18...",https://www.amazon.in/gp/slredirect/picassoRed...
1,LG,Gram 10th Gen Intel Core i5-1035G7 14-inch IP...,3.8 out of 5 stars,10 ratings,"₹ 67,990.00",7 Days Replacement,"Sunday, Feb 28",In stock.,"Brand LG---Manufacturer LG, 1800 315 9999 / 18...",https://www.amazon.in/gp/slredirect/picassoRed...
2,HP,15 Intel Pentium Gold 6405U Processor Entry L...,4.0 out of 5 stars,279 ratings,"₹ 26,990.00",7 Days Replacement,"Monday, March 1",In stock.,"Brand HP---Manufacturer hp, HP India Sales Pvt...",https://www.amazon.in/HP-Pentium-Processor-15-...
3,Lenovo,IdeaPad Slim 3i Intel Celeron N4020 15.6-inch...,3.4 out of 5 stars,4 ratings,"₹ 23,990.00",7 Days Replacement,"Sunday, Feb 28",In stock.,"Brand Lenovo---Manufacturer Lenovo, One of the...",https://www.amazon.in/Lenovo-IdeaPad-15-6-inch...
4,Lenovo,IdeaPad Slim 3 AMD Athlon Silver 3050U 15.6 i...,3.1 out of 5 stars,5 ratings,"₹ 25,990.00",7 Days Replacement,"Tuesday, March 2",In stock.,"Brand Lenovo---Manufacturer Lenovo, One of the...",https://www.amazon.in/Lenovo-Integrated-Graphi...
...,...,...,...,...,...,...,...,...,...,...
61,Dell,"Inspiron 3505 15.6"" HD AG Display Laptop (Ath...",5.0 out of 5 stars,1 rating,"₹ 29,797.00",7 Days Replacement,"Sunday, Feb 28",In stock.,"Brand Dell---Manufacturer Dell India Pvt Ltd, ...",https://www.amazon.in/Dell-Inspiron-Integrated...
62,Asus,VivoBook 14 Core i3 8th Gen - (4 GB/512 GB SS...,-,-,"₹ 38,990.00",7 Days Replacement,Feb 27 - March 2,Only 1 left in stock.,"Brand ASUS---Manufacturer Asus, ASUS---Series ...",https://www.amazon.in/Asus-VivoBook-Core-8th-X...
63,HP,14 10th Gen Intel Core i5 14-inch HD Laptop (...,3.8 out of 5 stars,234 ratings,"₹ 47,490.00",7 Days Replacement,Feb 27 - March 1,Available to ship in 1-2 days.,"Brand HP---Manufacturer hp, HP India Sales Pvt...",https://www.amazon.in/HP-14-inch-Laptop-i5-102...
64,Lenovo,Ideapad Slim 3i Intel Core i5 10th Gen 15.6 F...,3.4 out of 5 stars,517 ratings,"₹ 50,990.00",7 Days Replacement,"Sunday, Feb 28",In stock.,"Brand Lenovo---Manufacturer Lenovo, Hefei Bitl...",https://www.amazon.in/gp/slredirect/picassoRed...


#### Question 3 - Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [5]:
#load the URL
driver=webdriver.Chrome(r"C:\Users\rahsh\Downloads\chromedriver_win32\chromedriver.exe")
url=('https://images.google.com/')
driver.get(url)
driver.implicitly_wait(2)

In [6]:
#Define a function which accepts image name as an argument
def get_urls(keywords):
        driver.get(url)
        
        #Let's enter the details in the search column and click search button
        search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search_bar.send_keys(keywords)
        button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
        button.click()
        driver.implicitly_wait(5)
        
        
#load few pages to scrape images from them
        for k in range(12):
            driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
        picture=[imgage for imgage in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        image_urls=[]
        print(len(picture))
        if(len(picture)==100):
            for i in picture:
                try:
                    i.click()
                    image_urls.append(i.get_attribute('href'))
                except:
                    image_urls.append('Not Available')
            return image_urls
        else:
            print('Image Not Available')

In [7]:
#scrape the url's of all images
fruits=get_urls('Fruits')
cars=get_urls('Cars')
machine_learning=get_urls('Machine Learning')

100
100
100


In [8]:
#create a DataFrame for our data
Image_URL=pd.DataFrame({})
Image_URL['Fruits']=fruits
Image_URL['Cars']=cars
Image_URL['Machine Learning']=machine_learning
Image_URL

,Fruits,Cars,Machine Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...


#### Question 4 - Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [9]:
#loading the drivers and URL
driver=webdriver.Chrome(r"C:\Users\rahsh\Downloads\chromedriver_win32\chromedriver.exe")
driver.get('https://www.flipkart.com/')
driver.implicitly_wait(3)

#enter the details in the search column
product=input('Enter the product vertival that needs to be searched : ')
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
driver.implicitly_wait(3)
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(product))
driver.find_element_by_xpath('//button[@type="submit"]').click()
driver.implicitly_wait(3)

Enter the product vertival that needs to be searched : smartphone


In [10]:
#create an empty list to store the scrape data
product_urls=[]
brand_name=[]
smartphone_name=[]
smartphone_color=[]
RAM=[]
ROM=[]
primary_camera=[]
secondary_camera=[]
display_size=[]
display_resolution=[]
processor=[]
processor_core=[]
battery_capacity=[]
price=[]


links=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

#scrape product URL
for i in links:
    product_urls.append(i.get_attribute('href'))
for i in product_urls:
    driver.get(i)
    driver.implicitly_wait(4)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        
#scrape brand name
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        brand_name.append(title[0])
    except:
        brand_name.append('-')
    try:
        
#scrape smartohone name
        smartphone_name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        smartphone_name.append('-')
    try:
        
#scrape smartphone colour
        smartphone_color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        smartphone_color.append('-')
    try:
        
#scrape smartphone RAM
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        RAM.append('-')
    try:
        
#scrape smartphone ROM
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        ROM.append('-')
    try:
        
#scrape smartphone primary camera details
        primary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        primary_camera.append('-')
    try:
        
#scrape smartphone secondary camera details
        secondary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        secondary_camera.append('-')
    try:
        
#scrape smartphone display size
        display_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        display_size.append('-')
    try:
        
#scrape smartphone screen resolution
        display_resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        display_resolution.append('-')
    try:
        
#scrape smartphone processor
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        processor.append('-')
    try:
        
#scrape smartphone processor core
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        processor_core.append('-')
    try:
        
#scrape smartphone battery capacity
        battery_capacity.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        battery_capacity.append('-')
    try:
        
#scrape smartphone price
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')

In [11]:
#create a DataFrame for our data
flipkart=pd.DataFrame({})
flipkart['Brand Name']=brand_name
flipkart['Smartphone Name']=smartphone_name
flipkart['Smartphone Colour']=smartphone_color
flipkart['RAM']=RAM
flipkart['Storage (ROM)']=ROM
flipkart['Primray Camera']=primary_camera
flipkart['Secondary Camera']=secondary_camera
flipkart['Display Size']=display_size
flipkart['Display Resolution']=display_resolution
flipkart['Processor']=processor
flipkart['Processor Cores']=processor_core
flipkart['Battery Capacity']=battery_capacity
flipkart['Price']=price
flipkart['Product URL']=product_urls
flipkart

,Brand Name,Smartphone Name,Smartphone Colour,RAM,Storage (ROM),Primray Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product URL
0,POCO,C3,Arctic Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,1 Year on Handset and 6 Months on Accessories,"₹6,999",https://www.flipkart.com/poco-c3-arctic-blue-3...
1,POCO,C3,Lime Green,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,1 Year on Handset and 6 Months on Accessories,"₹6,999",https://www.flipkart.com/poco-c3-lime-green-32...
2,POCO,M3,Cool Blue,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,77.3 mm,"₹10,999",https://www.flipkart.com/poco-m3-cool-blue-64-...
3,POCO,C3,Arctic Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,1 Year on Handset and 6 Months on Accessories,"₹7,999",https://www.flipkart.com/poco-c3-arctic-blue-6...
4,Redmi,9i,Midnight Black,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹7,999",https://www.flipkart.com/redmi-9i-midnight-bla...
5,POCO,M3,Power Black,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,77.3 mm,"₹10,999",https://www.flipkart.com/poco-m3-power-black-6...
6,Infinix,Smart HD 2021,Quartz Green,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,15.49 cm (6.1 inch),1560 x 720 Pixels,MediaTek Helio A20,Quad Core,5000 mAh,"₹6,199",https://www.flipkart.com/infinix-smart-hd-2021...
7,POCO,M3,Yellow,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,77.3 mm,"₹10,999",https://www.flipkart.com/poco-m3-yellow-64-gb/...
8,Redmi,Redmi 9 Prime,Space Blue,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-space-b...
9,Realme,Narzo 20A,Glory Silver,4 GB,64 GB,12MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Qualcomm Snapdragon 665,Octa Core,5000 mAh,"₹9,499",https://www.flipkart.com/realme-narzo-20a-glor...


In [12]:
#save the dataframe in csv file
flipkart.to_csv('flipkart_smartphone.csv')

#### Question 5 - Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [13]:
#load the drivers and URL
driver=webdriver.Chrome(r"C:\Users\rahsh\Downloads\chromedriver_win32\chromedriver.exe")
driver.maximize_window()
driver.get('https://www.google.com/maps/')
driver.implicitly_wait(5)

#create a function to enter the city name 
city = input('Enter City Name : ') 
driver.find_element_by_id('searchboxinput').send_keys(city)
    
#create a fucntion to search the result
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
driver.implicitly_wait(6)

#Scrape the entered city latitude and Longitude from google map
lat=driver.current_url.split('/@')[1].split(',')[0]
long=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',lat,'Longitude:',long)

Enter City Name : Nanital
Latitude: 20.9880135 Longitude: 82.7525294


#### Question 6 - Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [14]:
#load the drivers and URL
driver=webdriver.Chrome(r"C:\Users\rahsh\Downloads\chromedriver_win32\chromedriver.exe")
driver.get('https://trak.in/')
driver.implicitly_wait(5)

In [15]:
#enter the details in the search column
search_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(search_button)

In [16]:
#create an empty list to store data
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [17]:
#define a function to get the details
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
#scrape the date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
#scrape the detaisl
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup.append(n.text)
    
#scrape the industry details
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)
    
#scrape the subvertical details
    subvertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in subvertical:
        SubVertical.append(s.text)

#scrape the location details
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
#scrape the investors detils
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
    
#scrape the investment details
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
#scrape the amount
    amtount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amtount:
        Amount.append(a.text)

In [18]:
#create a DataFrame for our data
trak_fund=pd.DataFrame({})
trak_fund['Date']=Date
trak_fund['Startup']=Startup 
trak_fund['Industry']=Industry
trak_fund['SubVertical']=SubVertical
trak_fund['Location']=Location
trak_fund['Investor']=Investor
trak_fund['Investment']=Investment
trak_fund['Amount']=Amount

In [19]:
trak_fund

,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


#### Question 7 - Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [20]:
#load the drivers and URL
driver=webdriver.Chrome(r"C:\Users\rahsh\Downloads\chromedriver_win32\chromedriver.exe")
driver.get('https://www.digit.in/')
driver.implicitly_wait(3)

In [21]:
# scroll through menubar and click on top10
driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()
time.sleep(3)

#find the best gaming laptop in India and click it
driver.find_element_by_xpath('//div[@class="listing_container"]/ul/li[26]/a').click()

In [22]:
#create an empty list to store data
name = []
OS = []
display = []
processor = []
HDD = []
price = []
RAM = []
dimension = []
GPU = []
weight = []

#scrape the Company Name
Name=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in Name:
    name.append(i.text)
    
#scrape the OS type
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#scrape the display size
Display=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in Display:
    display.append(i.text)
    
# scrape the processor type
Processor=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in Processor:
    processor.append(i.text)
processor

#scrape the Memory
Memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")
Memory_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]") 
for i in range(len(Memory)):
        if Memory[i].text=='Memory':
            HDD.append(Memory_spec[i].text.split('/')[0])
            RAM.append(Memory_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')
            RAM.append('No details available')

#scrape the weight
Weight=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
Weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(Weight)):
        if Weight[i].text=='Weight':
            weight.append(Weight_spec[i].text)
        
#scrape the dimensions
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#scrape the Graphic Processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)

#scrape the other details 
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))

#create a for loop to scrape the details from each page
for i in full_specs:
    driver.get(i)
    try:
        Price=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(Price.text)
    except NoSuchElementException:
        price.append("No details available")

#create a DataFrame for our data
gaming_laptop=pd.DataFrame({"Campany Name":name,"OS":OS,"Screen Display Size":display,
                "HDD":HDD,"RAM":RAM,"Processor":processor,
                "Weight":weight,"Dimension (mm)":dimension,
                "Graphical Processor":GPU,"Price (Rs.)":price})

In [23]:
gaming_laptop

,Campany Name,OS,Screen Display Size,HDD,RAM,Processor,Weight,Dimension (mm),Graphical Processor,Price (Rs.)
0,MSI GT76 TITAN DT 9SG,WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",1 TB HDD,64 GBGB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,"379,990"
1,ALIENWARE 17 AREA-51M,WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",1 TB PCIe SSD,32GB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,"489,909"
2,HP OMEN 15 2020,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16 GBGB DDR4,INTEL I7-10750H 10TH GEN | 1.6GHZ,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,"114,990"
3,ASUS ZEPHYRUS G14,WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"164,990"
4,LENOVO LEGION Y540,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,8GB DDR4,9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,"79,990"
5,ASUS ROG ZEPHYRUS G GA502,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16GB DDR4,AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,"79,990"
6,ASUS ROG ZEPHYRUS S GX531,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512GB SSD,16 GBGB DDR4,9TH GEN INTEL CORE I7-8750H | 2.2 GHZ,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),"146,000"
7,MSI GT83VR 7RE TITAN SLI,WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",1.5 TB SATA,64GB DDR4,INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ,5.5,458 x 339 x 69,Dual GTX1070,"349,990"
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"259,990"
9,DELL G3,WINDOWS 10 HOME PLUS,15.6 MP | NA,256 + 1 TB SSD + SATA,16GB DDR4,8TH GENERATION INTEL® CORE™ I7-8750H | NA,2.53,22.7 X 380 X 258,NVIDIA® GeForce® GTX,"61,990"
